In [1]:
import pandas as pd
import mysql.connector
import numpy as np

In [7]:
All_bus_info_to_sql = pd.read_csv("All_bus_info1")

In [12]:
All_bus_info_to_sql['Ratings'] = All_bus_info_to_sql['Ratings'].str.replace(r'\n\d+', '', regex=True).str.strip()

In [14]:
All_bus_info_to_sql["Ratings"].dtype()

,Bus Name,Bus Type,Start Time,End Time,Duration,Price,Seat Availability,Ratings,Route Name,Route Link,State
0,NBSTC -SLG-ROH-DJL-01-7:00 - 161760,Non AC Seater 2+2,07:00,10:00,03h 00m,INR 105,10 Seats available,3.2,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...,North Bengal
1,NBSTC -SLG-ROH-DJL-02-8:00 - 158470,Non AC Seater 2+2,08:00,11:00,03h 00m,INR 105,4 Seats available,3.7,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...,North Bengal


In [10]:
##sql connection 
connection = mysql.connector.connect(
    host ="localhost",
    database = "Redbus_information",
    user="root",
    password="1234",
    port="3306"
)
cursor = connection.cursor()

In [11]:
cursor.execute("DROP TABLE IF EXISTS REDBUS_TABLE")

In [15]:
#TABLE CREATION
create_query = '''CREATE TABLE IF NOT EXISTS REDBUS_TABLE(
    ID INT AUTO_INCREMENT PRIMARY KEY, 
    BUS_NAME VARCHAR(100), 
    BUS_TYPE TEXT, 
    START_TIME TIME, 
    END_TIME TIME, 
    DURATION VARCHAR(10), 
    PRICE DECIMAL(10, 2), 
    SEAT_AVAILABILITY INT, 
    RATINGS VARCHAR(10), 
    ROUTE_NAME VARCHAR(200),
    ROUTE_LINK TEXT,
    STATE TEXT
)'''

cursor.execute(create_query)
connection.commit()

In [ ]:
#CLEANING DATA
# Replace NaN values with None
#All_bus_info_to_sql = All_bus_info_to_sql.where(pd.notnull(All_bus_info_to_sql), None)

# Clean the 'Price' column: Remove 'INR ' and convert to float
All_bus_info_to_sql['Price'] = All_bus_info_to_sql['Price'].astype(str).str.replace('INR ', '').astype(float)





In [28]:
# Clean the 'Seat Availability' column: Replace empty strings and non-digit characters, then convert to integer
All_bus_info_to_sql['Seat Availability'] = (
    All_bus_info_to_sql['Seat Availability']
    .replace('', '0')  # Replace empty strings with '0'
    .fillna('0')  # Replace NaN with '0'
    .astype(str)  # Convert to string to allow regex replacement
    .str.replace(r'\D', '', regex=True)  # Remove non-digit characters
    .replace('', '0')  # Replace any remaining empty strings with '0'
    .astype(int)  # Convert to integer
)


In [29]:
All_bus_info_to_sql.head()

,Bus Name,Bus Type,Start Time,End Time,Duration,Price,Seat Availability,Ratings,Route Name,Route Link,State
0,NBSTC -SLG-ROH-DJL-01-7:00 - 161760,Non AC Seater 2+2,07:00,10:00,03h 00m,105.0,10,3.2,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...,North Bengal
1,NBSTC -SLG-ROH-DJL-02-8:00 - 158470,Non AC Seater 2+2,08:00,11:00,03h 00m,105.0,4,3.7,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...,North Bengal
2,NBSTC-SLG-KOL-AC-18:45 - 176648,AC Seater (2+2),18:45,07:00,12h 15m,1245.0,0,5.0,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,North Bengal
3,NBSTC-SLG-KOL-AC-19:00 - 157844,AC Seater (2+2),19:00,07:00,12h 00m,1245.0,0,2.9,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,North Bengal
4,WBTC (CTC) KOLKATA-RAIGUNGE-SILIGURI(AC-VOLVO)...,Volvo AC Seater (2+2),18:30,06:15,11h 45m,1355.0,0,3.5,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,North Bengal


In [30]:
#DATA INSERTION INTO TABLE 
insert_query ='''INSERT INTO REDBUS_TABLE(BUS_NAME ,BUS_TYPE ,START_TIME ,END_TIME ,DURATION ,PRICE ,SEAT_AVAILABILITY ,RATINGS ,
                                        ROUTE_NAME ,ROUTE_LINK ,STATE ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
data = All_bus_info_to_sql.values.tolist()
cursor.executemany(insert_query,data)
connection.commit() 

In [23]:
has_nan = All_bus_info_to_sql.isnull().sum()
print(has_nan)


Bus Name             0
Bus Type             0
Start Time           0
End Time             0
Duration             0
Price                0
Seat Availability    0
Ratings              0
Route Name           0
Route Link           0
State                0
dtype: int64


In [22]:
All_bus_info_to_sql.fillna(value="Nothing to display", inplace=True)


In [16]:
data = All_bus_info_to_sql.values.tolist()
data[0]

['NBSTC -SLG-ROH-DJL-01-7:00 - 161760',
 'Non AC Seater 2+2',
 '07:00',
 '10:00',
 '03h 00m',
 'INR 105',
 '10 Seats available',
 '3.2',
 'Siliguri to Darjeeling',
 'https://www.redbus.in/bus-tickets/siliguri-to-darjeeling',
 'North Bengal']

In [17]:
All_bus_info_to_sql["Bus Type"]

0                    Non AC Seater 2+2
1                    Non AC Seater 2+2
2                      AC Seater (2+2)
3                      AC Seater (2+2)
4                Volvo AC Seater (2+2)
                     ...              
5450                 A/C Sleeper (2+1)
5451    Non A/C Seater / Sleeper (2+1)
5452    Non A/C Seater / Sleeper (2+1)
5453                 A/C Sleeper (2+1)
5454    Non A/C Seater / Sleeper (2+1)
Name: Bus Type, Length: 5455, dtype: object